[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

In [ ]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [ ]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("cases_and_deaths.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

In [ ]:


def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """

    listofdata=jsondata['data']
    
    dates=[dictionary['date'] for dictionary in listofdata ]
    dates.sort()
    
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")
    
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    print (startdate, ' to ', enddate)
    
    index=pd.date_range(startdate, enddate, freq='D')
    casesanddeathes=pd.DataFrame(index=index, columns=['newCases', 'cumCases', 'newDeaths28Days', 'cumDeaths28Days'])
    
    for entry in listofdata: 
        date=parse_date(entry['date'])
        for column in ['newCases', 'cumCases', 'newDeaths28Days', 'cumDeaths28Days']:
            if pd.isna(casesanddeathes.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                casesanddeathes.loc[date, column]=value
            
    casesanddeathes.fillna(0.0, inplace=True)
    
    
    return casesanddeathes

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
casesanddeathes=wrangle_data(jsondata) # df is the dataframe for plotting

In [ ]:
filters = [
    'areaType=nation',
    'areaName=Wales']

structure = {
    "date": "date",
   "newCases": "newCasesByPublishDate",
    "cumCases": "cumCasesByPublishDate",
    "newDeaths28Days": "newDeaths28DaysByDeathDate",
    "cumDeaths28Days": "cumDeaths28DaysByDeathDate"}

def access_api():
    api = Cov19API(filters=filters, structure=structure)
    cases_and_deaths=api.get_json()
    return cases_and_deaths

click the botton to refresh the data from Public Health England (PHE).

In [ ]:
# our API access function. This will be called by the button when it is clicked
def api_button_callback(button):
    
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
   
    apidata=access_api()

    global casesanddeathes
    casesanddeathes=wrangle_data(apidata)
   
    #refresh_graph()
    
    apibutton.icon="check"
    apibutton.disabled=True
# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(api_button_callback)

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets
display(apibutton)

## Graphs and Analysis

In [ ]:
casesanddeathes=pd.read_pickle("casesanddeathes.pkl")
series=wdg.SelectMultiple(
    options=['newCases', 'cumCases', 'newDeaths28Days', 'cumDeaths28Days'],
    value=['newCases', 'cumCases', 'newDeaths28Days', 'cumDeaths28Days'],
    rows=4,
    description='Stats:',
    disabled=False)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False)

controls=wdg.HBox([series, scale])

def casesanddeaths_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        casesanddeathes[list(gcols)].plot(logy=logscale)
        plt.show() 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

graph=wdg.interactive_output(casesanddeaths_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)